## Final Code with deatils

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

## HyperParameters

In [ ]:
# Hyperparameters
batch_size = 64
block_size = 256

max_iters = 5000  # to be tuned later
eval_interval = 500
learning_rate = 3e-4

device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
head_size = 16
dropout = 0.2
# ------------------

## Dataset
We're using the same Shakespere dataset, however A larger dataset such as [OpenWebText](https://openwebtext2.readthedocs.io/en/latest/) can be used.

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-30 20:02:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2023-11-30 20:02:09 (25.7 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
# from datasets import load_dataset
# dataset = load_dataset("Skylion007/openwebtext")

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = ['\n',' ', '!', '$', '&', '\'', ',', '-', '.', '3', ':', ';', '?', 'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
vocab_size = len(chars)



### **Encoding/Decoding**


In [ ]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### Data Loading

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Self Attention (with RoPE)
Rotary Positional Embedding

In [ ]:
!pip install rotary-embedding-torch

In [ ]:
import torch
from rotary_embedding_torch import RotaryEmbedding

# rotary_emb = RotaryEmbedding(dim = 32)

In [ ]:
class Head(nn.Module):

    def __init__(self, head_size, RoPE=False):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)
        self.rope = RoPE
        if RoPE:
          self.rotary_emb = RotaryEmbedding(dim=head_size)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # Applying rotary ----------------------
        if self.rope:
            q = self.rotary_emb.rotate_queries_or_keys(q)
            k = self.rotary_emb.rotate_queries_or_keys(k)
        # --------------------------------------
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

### GPT Model with both token and position embedding

In [ ]:
class GPT(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


### Running and Evaluating

In [ ]:
model = GPT()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# Using ADAM optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluating the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)


10.788929 M parameters
step 0: train loss 4.3331, val loss 4.3317
step 500: train loss 2.0315, val loss 2.1073
step 1000: train loss 1.6151, val loss 1.7852
step 1500: train loss 1.4533, val loss 1.6504
step 2000: train loss 1.3522, val loss 1.5616
step 2500: train loss 1.2832, val loss 1.5238
step 3000: train loss 1.2356, val loss 1.5053
step 3500: train loss 1.1899, val loss 1.4925
step 4000: train loss 1.1564, val loss 1.4766
step 4500: train loss 1.1183, val loss 1.4763
step 4999: train loss 1.0849, val loss 1.4788


In [ ]:
print(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))




HENRY BOLINGBROKE:
Then, grave me my lord's uncle;
I never true Gremio's dauble king make make his wrongly
Is this my way: a playforment stretched,
What to Walesby, poisoner to England;
Spreved his father Richard lies kill dispraughter.
The orfer is the nor Edward, and come
To Raves Nortespirah?
My Lord's rest Christsengthman: be defiged
Tim haply but promised to the motheror;
Thus had stew us I losen to thy stains.
Death, there he canst thou a bride!

BENTIO:
Take hee them.

BISHOP OF WARWISLOUCESTER:
That you, friend me not play to quicker.

QUEEN:
Would told Come hither weary well, Claesby,
Her robes and the England comest counselected their affairs.
The venom heavy manners proceeance in joy,
I am hairy to breach and Montague.

Lord Mayor:
What's that, are not yet to lent the gentlay tame,
They comes the tenderous place and traves.

Second Murderer:
To exerer.

AEdile:

Civenture, Clifford:
You better son, the first uphurp of herself:
He shall be seasonly loves.
We may not arm, ev

### Running DDP (Distributed Data Parallel)
```(code will only work on multiple GPUs)```

In [ ]:
import os
import sys
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'

    # initialize the process group
    dist.init_process_group("gloo", rank=rank, world_size=world_size)


def cleanup():
  dist.destroy_process_group()

In [ ]:
def training_loop(rank, world_size):
  setup(rank, world_size)
  print(f'RANK: {rank}')
  model = GPT().to(rank)
  ddp_model = DDP(model, device_ids=[rank], output_device=rank)
  optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

  for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = ddp_model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
  cleanup()

def run_function(runner_fn, world_size):
  mp.spawn(runner_fn, args=(world_size),
           nprocs=world_size)

In [ ]:
world_size = 1  # asssuming we have 1 GPUs
run_function(training_loop, world_size)